# 🧠 Sentium — Colab Training Notebook
**Samuel Tanaka · Universitas Indonesia · February 2026**

This notebook trains the **Sentium Phase 0 baseline (454 M params)** on Google Colab using your GPU allocation, with all checkpoints and datasets living in your **2 TB Google Drive**.

### Workflow
```
GitHub repo (code)  ──►  Colab VM (compute)  ──►  Google Drive (data & checkpoints)
      ▲                                                      │
      └──────── push from VS Code ───────────────────────────┘
                                           resume next session ◄─┘
```

### Sections
1. Mount Google Drive
2. GPU / runtime check
3. Install dependencies
4. Clone / pull repo from GitHub
5. Configure Drive paths
6. Load dataset (HuggingFace streaming → Drive cache)
7. Build model
8. Configure checkpoint saving to Drive
9. Train
10. Resume training from checkpoint

In [ ]:
!sudo apt-get update -y && sudo apt-get full-upgrade -y

In [2]:
!python --version

Python 3.12.12


## 1 · Mount Google Drive
Your 2 TB Drive is the persistent storage layer — checkpoints survive session restarts, and the dataset cache means you won't re-download every time.

In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

import os
DRIVE_ROOT = "/content/drive/MyDrive/Sentium"
os.makedirs(DRIVE_ROOT, exist_ok=True)
print(f"✅ Drive mounted. Working root: {DRIVE_ROOT}")
print(f"   Free space: ", end="")
os.system(f"df -h /content/drive | tail -1 | awk '{{print $4}}'")


Mounted at /content/drive
✅ Drive mounted. Working root: /content/drive/MyDrive/Sentium
   Free space: ✅ Drive mounted. Working root: /content/drive/MyDrive/Sentium
   Free space: 

0

## 2 · GPU / Runtime Check
Verify what hardware Colab allocated. **If you see CPU here, go to Runtime → Change runtime type → GPU.**

In [4]:
import torch

if torch.cuda.is_available():
    props = torch.cuda.get_device_properties(0)
    vram_gb = props.total_memory / 1024**3
    print(f"✅ GPU: {props.name}")
    print(f"   VRAM: {vram_gb:.1f} GB")
    print(f"   CUDA: {torch.version.cuda}")
    print(f"   cuDNN: {torch.backends.cudnn.version()}")
    # Colab GPU tier suggestion
    if vram_gb >= 40:
        tier = "A100 🚀  (best tier — batch=16, no grad-ckpt)"
    elif vram_gb >= 22:
        tier = "L4 ⚡  (great — batch=8)"
    elif vram_gb >= 15:
        tier = "T4 ✔  (standard — batch=4)"
    else:
        tier = "unknown / limited"
    print(f"   Tier: {tier}")
else:
    print("❌ No GPU detected. Go to Runtime → Change runtime type → GPU")

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader 2>/dev/null || echo "(nvidia-smi not available)"


✅ GPU: Tesla T4
   VRAM: 14.6 GB
   CUDA: 12.6
   cuDNN: 91002
   Tier: unknown / limited
Tesla T4, 15360 MiB, 14910 MiB


## 3 · Install Dependencies
`transformers` and `datasets` are needed for real corpus training. `wandb` is optional but recommended for loss curves you can view from your laptop.

In [19]:
%%capture install_output
# PyTorch is pre-installed on Colab — only install the extras
!pip install -q \
    einops>=0.7.0 \
    transformers>=4.40.0 \
    datasets>=2.18.0 \
    tokenizers>=0.19.0 \
    wandb>=0.17.0 \
    tqdm>=4.66.0 \
    scipy>=1.12.0

print("✅ Dependencies installed.")


In [22]:
!pip install jedi>=0.16.0

## 4 · Clone / Update Repository from GitHub
The code lives on GitHub. Every time you open a new Colab session, this cell pulls the latest version — so your edits from VS Code are instantly available here after a `git push`.

In [23]:
import os

# ─── CONFIGURE THIS ────────────────────────────────────────────────────────
GITHUB_USER = "sammtan"   # ← change this
REPO_NAME   = "Sentia"                 # ← change if your repo name differs
BRANCH      = "master"
# If repo is private, use a Personal Access Token (PAT):
#   GITHUB_TOKEN = "ghp_xxxxxxxxxxxx"
#   REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git"
REPO_URL    = f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git"
# ───────────────────────────────────────────────────────────────────────────

REPO_DIR = f"/content/{REPO_NAME}"

if os.path.exists(REPO_DIR):
    print(f"📦 Repo already cloned — pulling latest changes from '{BRANCH}'...")
    !git -C {REPO_DIR} fetch origin
    !git -C {REPO_DIR} reset --hard origin/{BRANCH}
else:
    print(f"📦 Cloning {REPO_URL} ...")
    !git clone --branch {BRANCH} {REPO_URL} {REPO_DIR}

# Upgrade setuptools first — Colab's system version is too old for editable installs
!pip install -q --upgrade setuptools wheel

# Install the sentium package in editable mode
!pip install -q -e {REPO_DIR}[train]

# Add repo to path so imports work
import sys
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

print(f"✅ Repo ready at {REPO_DIR}")
!git -C {REPO_DIR} log --oneline -3


📦 Repo already cloned — pulling latest changes from 'master'...
HEAD is now at a3d6778 fix: use setuptools.build_meta backend, upgrade setuptools in Colab before editable install
HEAD is now at a3d6778 fix: use setuptools.build_meta backend, upgrade setuptools in Colab before editable install
  Installing build dependencies ... done
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sentium (pyproject.toml) ... done
  Building editable for sentium (pyproject.toml) ... done
✅ Repo ready at /content/Sentia
a3d6778 (HEAD -> master, origin/master, origin/HEAD) fix: use setuptools.build_meta backend, upgrade setuptools in Colab before editable insta

In [ ]:
!python --version

## 5 · Configure Drive Storage Paths
All persistent data (datasets cache, checkpoints, logs) goes into **MyDrive/Sentium/** — survives session disconnects.

In [24]:
from pathlib import Path

# ─── All persistent storage lives here ────────────────────────────────────
DRIVE_ROOT       = Path("/content/drive/MyDrive/Sentium")
DRIVE_CKPT_DIR   = DRIVE_ROOT / "checkpoints"      # .pt files
DRIVE_CACHE_DIR  = DRIVE_ROOT / "datasets_cache"   # HuggingFace cache
DRIVE_LOG_DIR    = DRIVE_ROOT / "logs"             # training logs
# ───────────────────────────────────────────────────────────────────────────

# Local (VM) checkpoint dir — synced to Drive after each save
LOCAL_CKPT_DIR   = Path("/content/checkpoints")

for d in [DRIVE_CKPT_DIR, DRIVE_CACHE_DIR, DRIVE_LOG_DIR, LOCAL_CKPT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Drive paths configured:")
for name, path in {
    "Checkpoints": DRIVE_CKPT_DIR,
    "Dataset cache": DRIVE_CACHE_DIR,
    "Logs": DRIVE_LOG_DIR,
}.items():
    size = sum(f.stat().st_size for f in path.rglob("*") if f.is_file())
    print(f"  {name:16s}: {path}   ({size/1024**3:.2f} GB used)")


Drive paths configured:
  Checkpoints     : /content/drive/MyDrive/Sentium/checkpoints   (0.00 GB used)
  Dataset cache   : /content/drive/MyDrive/Sentium/datasets_cache   (0.00 GB used)
  Logs            : /content/drive/MyDrive/Sentium/logs   (0.00 GB used)


## 6 · Load Dataset from Drive Cache
Uses HuggingFace `datasets` in **streaming mode** — data flows token-by-token from the internet and is cached on Drive so subsequent sessions are instant.

> **Switch dataset** by changing `DATASET_ID` below.
>
> | Alias | HuggingFace ID | Size |
> |---|---|---|
> | `slim_pajama` | `DKYoon/SlimPajama-6B` | ~6 B tokens, 5.5 M rows |
> | `the_pile` | `monology/pile-uncopyrighted` | ~800 GB, unrestricted |
> | `openwebtext` | `Skylion007/openwebtext` | ~38 GB |
> | `wikitext` | `wikitext-103-raw-v1` | ~500 MB (good for smoke tests) |
>
> You can also pass any raw HuggingFace dataset id (e.g. `"allenai/c4"`).

In [25]:
import sys
sys.path.insert(0, REPO_DIR)  # ensure sentium is importable

from train_colab import HFTextDataset, RandomTokenDataset
from torch.utils.data import DataLoader

# ─── Dataset config ────────────────────────────────────────────────────────
DATASET_ID     = "slim_pajama"  # slim_pajama | the_pile | openwebtext | wikitext
TOKENIZER_NAME = "gpt2"
CONTEXT_LEN    = 512            # will be overridden by train_cfg below
MAX_SAMPLES    = None           # set an int (e.g. 50_000) to cap stream for testing
# ───────────────────────────────────────────────────────────────────────────

print(f"Loading dataset: {DATASET_ID}")
print(f"Tokenizer:       {TOKENIZER_NAME}")
print(f"Cache dir:       {DRIVE_CACHE_DIR}")
print("(First run downloads and caches — subsequent runs load from Drive instantly)")
# Dataset is created lazily (streaming) — actual loading starts during training


Loading dataset: slim_pajama
Tokenizer:       gpt2
Cache dir:       /content/drive/MyDrive/Sentium/datasets_cache
(First run downloads and caches — subsequent runs load from Drive instantly)


## 7 · Build Model
Auto-selects VRAM-safe `TrainConfig` for whatever GPU Colab gave you.

In [26]:
import torch
from sentium import Sentium, SentiumConfig
from sentium.train import TrainConfig
from train_colab import detect_gpu_vram, vram_safe_train_cfg

# ── Auto-detect VRAM and pick matching config ──────────────────────────────
detected_vram = detect_gpu_vram()

# ─── Training config (auto-selected; override below if needed) ─────────────
train_cfg = vram_safe_train_cfg(
    vram_gb=detected_vram,
    save_dir=str(LOCAL_CKPT_DIR),
    wandb=False,            # set True after wandb.login() below
)
# ── Optional manual overrides ──────────────────────────────────────────────
# train_cfg.max_steps      = 200_000
# train_cfg.batch_size     = 8
# train_cfg.context_end    = 2048
# ───────────────────────────────────────────────────────────────────────────

# Match dataset context length to training config
CONTEXT_LEN = train_cfg.context_start

# ── Model ──────────────────────────────────────────────────────────────────
model_cfg = SentiumConfig.baseline_200m()
model     = Sentium(model_cfg)

params_m = model.num_parameters() / 1e6
print(f"✅ Model: {model_cfg.model_name}  —  {params_m:.1f} M parameters")
print(f"   d_model={model_cfg.d_model}, n_layers={model_cfg.n_layers}, "
      f"n_heads={model_cfg.n_heads}, d_ff={model_cfg.d_ff}")
print(f"\nTraining config (VRAM={detected_vram:.1f} GB):")
print(f"  batch_size            = {train_cfg.batch_size}")
print(f"  grad_accum            = {train_cfg.grad_accum}")
print(f"  effective_batch       = {train_cfg.batch_size * train_cfg.grad_accum}")
print(f"  context_start         = {train_cfg.context_start}")
print(f"  context_end           = {train_cfg.context_end}")
print(f"  gradient_checkpointing= {train_cfg.gradient_checkpointing}")
print(f"  dtype                 = {train_cfg.dtype}")
print(f"  max_steps             = {train_cfg.max_steps:,}")


✅ Model: sentium-baseline-200m  —  454.2 M parameters
   d_model=1024, n_layers=24, n_heads=16, d_ff=4096

Training config (VRAM=15.0 GB):
  batch_size            = 4
  grad_accum            = 4
  effective_batch       = 16
  context_start         = 512
  context_end           = 4096
  gradient_checkpointing= False
  dtype                 = bf16
  max_steps             = 100,000


## 8 · Configure Checkpointing to Drive
Every `save_every` steps, a `.pt` file is written locally **and immediately copied to Drive**. The Drive copy is the ground truth — if Colab disconnects mid-run, you lose at most one interval.

In [27]:
from train_colab import patch_trainer_drive_sync, latest_checkpoint
from sentium.train import Trainer

# ─── Checkpoint interval (steps) ──────────────────────────────────────────
train_cfg.save_every  = 500    # save local + Drive every 500 steps
train_cfg.keep_last_n = 5      # keep 5 most recent checkpoints on Drive
# ───────────────────────────────────────────────────────────────────────────

# Build dataset + dataloader (streaming, Drive-cached)
dataset = HFTextDataset(
    dataset_id=DATASET_ID,
    seq_len=CONTEXT_LEN,
    cache_dir=str(DRIVE_CACHE_DIR),
    tokenizer_name=TOKENIZER_NAME,
    max_samples=MAX_SAMPLES,
)
loader = DataLoader(
    dataset,
    batch_size=train_cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

# Build trainer
trainer = Trainer(
    model=model,
    train_loader=loader,
    model_config=model_cfg,
    train_config=train_cfg,
    device="cuda",
)

# Patch Drive sync into every save call
patch_trainer_drive_sync(trainer, DRIVE_CKPT_DIR)

print(f"✅ Trainer ready.")
print(f"   Checkpoint every {train_cfg.save_every} steps → {DRIVE_CKPT_DIR}")
print(f"   Keeping last {train_cfg.keep_last_n} checkpoints")

# List existing checkpoints on Drive
existing = sorted(DRIVE_CKPT_DIR.glob("sentium_step*.pt"))
if existing:
    print(f"\n   Existing Drive checkpoints ({len(existing)}):")
    for p in existing[-5:]:
        size_mb = p.stat().st_size / 1024**2
        print(f"     {p.name}  ({size_mb:.0f} MB)")
else:
    print("\n   No existing checkpoints on Drive — fresh start.")


DatasetNotFoundError: Dataset 'cerebras/SlimPajama-627B' doesn't exist on the Hub or cannot be accessed.

## 9 · Train 🚀
Run the cell below to start training. Progress logs are printed to this cell **and** appended to `MyDrive/Sentium/logs/sentium_train.log`.

> ⚠️ Colab free tier disconnects after ~90 min idle or ~12 h. The checkpoint synced to Drive on the last `save_every` boundary is fully resumable — see Section 10.

In [ ]:
import logging, sys, time

# Route trainer prints through logging so they appear in both cell output and log file
log_path = DRIVE_LOG_DIR / "sentium_train.log"
handlers = [logging.StreamHandler(sys.stdout),
            logging.FileHandler(log_path, mode="a")]
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(message)s",
                    datefmt="%H:%M:%S",
                    handlers=handlers, force=True)

print(f"📝 Log file: {log_path}")
print(f"🚀 Starting training — {model.num_parameters()/1e6:.1f} M params on {torch.cuda.get_device_name(0)}")
print(f"   Max steps: {train_cfg.max_steps:,}  |  Checkpoint every: {train_cfg.save_every}")
print("-" * 70)

t_start = time.time()
trainer.train()
elapsed = (time.time() - t_start) / 60
print(f"\n✅ Training complete in {elapsed:.1f} min. Final step: {trainer.step}")


## 10 · Resume Training from Drive Checkpoint
Session expired? New Colab session? Run cells 1–7 to re-mount Drive, re-clone, re-build model — then run this cell to load the latest checkpoint and continue exactly where you left off.

In [ ]:
from train_colab import latest_checkpoint

# ── Find latest checkpoint on Drive ───────────────────────────────────────
ckpt_path = latest_checkpoint(DRIVE_CKPT_DIR)

if ckpt_path is None:
    print("❌ No checkpoint found in Drive. Run Section 9 first.")
else:
    size_mb = ckpt_path.stat().st_size / 1024**2
    print(f"✅ Found checkpoint: {ckpt_path.name}  ({size_mb:.0f} MB)")

    # Load into trainer (restores model weights, optimizer state, step counter)
    trainer.load_checkpoint(ckpt_path)
    print(f"   Resumed from step {trainer.step:,}")
    print(f"   Remaining steps: {train_cfg.max_steps - trainer.step:,}")

    # ── Continue training ──────────────────────────────────────────────────
    print("\n🚀 Resuming training...")
    trainer.train()
    print(f"\n✅ Done. Final step: {trainer.step:,}")


---
## 11 · (Optional) Weights & Biases — Monitor Loss from Your Laptop
W&B lets you watch training curves in real-time from any browser, even while Colab is running on Google's machine.

In [ ]:
import wandb

# Login once — stores key in Colab's secret manager for the session
wandb.login()   # will prompt for API key; get one free at wandb.ai

# Then enable W&B in train_cfg and re-run Section 8+9:
train_cfg.use_wandb   = True
train_cfg.project_name = "sentium"
print("✅ W&B enabled. Training metrics will stream to wandb.ai/your-username/sentium")


---
## 12 · (Optional) Smoke Test — Verify Everything Works Before Full Run
Runs 50 steps on random data with a tiny model. Should complete in < 30 seconds on any GPU.

In [ ]:
!python {REPO_DIR}/train_colab.py \
    --smoke-test \
    --drive-dir "{DRIVE_ROOT}" \
    --no-drive
